In [2]:
import pandas as pd

In [65]:
def custom_compare(x, y):
    if str(x) != str(y):
        raise RuntimeError(f'Ожидаемое значение: {y}. Фактическое: {x}')

# Предобработка данных

In [56]:
def parse_user_action_solution(line):
    """
    Принимает строку в обозначенном формате, а возвращает словарь.
    Ключами в словаре являются строки 'id', 'action' и 'time',
    а значениями — соответствующие ключам значения из обрабатываемой строки.
    
    Аргументы:
        line: Строка, которую необходимо превратить в словарь.
        
    Возвращаемое значение:
        Словарь в обозначенном формате.
    """
    
    diction = {'id': None,'action': None,'time': None} 
    line = line.strip('()')
    split_line = str.split(line, ',')
    diction['id'] = int(str.split(split_line[0], ':')[1].strip())
    diction['action'] = str.split(split_line[1], ':')[1].strip(" ''")
    diction['time'] = pd.to_datetime(str.split(split_line[2], "'")[3], format = "%Y-%m-%d %H:%M:%S")
    return diction

In [58]:
line = "('id': 1, 'action': 'search', 'time': '1997-12-31 23:23:59')"

In [62]:
parse_user_action_solution(line)

{'id': 1, 'action': 'search', 'time': Timestamp('1997-12-31 23:23:59')}

In [59]:
convert_line = line.strip('()')
split_line = str.split(convert_line, ',')
print(split_line)
print(str.split(split_line[0], ':')[1].strip())
print(str.split(split_line[1], ':')[1].strip(" ''"))
#print(pd.to_datetime(str.split(split_line[2], ':')[1]))
#print(convert_line)
time = pd.to_datetime(str.split(split_line[2], "'")[3], format = "%Y-%m-%d %H:%M:%S")
time

["'id': 1", " 'action': 'search'", " 'time': '1997-12-31 23:23:59'"]
1
search


Timestamp('1997-12-31 23:23:59')

In [60]:
diction = {'id': None,'action': None,'time': None}
diction['id'] = int(str.split(split_line[0], ':')[1].strip())
diction['action'] = str.split(split_line[1], ':')[1].strip(" ''")
diction['time'] = time
diction

{'id': 1, 'action': 'search', 'time': Timestamp('1997-12-31 23:23:59')}

In [63]:
def parse_user_action_test():
    example_1_line = "('id': 7281910, 'action': 'click', 'time': '2023-01-01 12:23:54')"
    example_1_res = {
        'id': 7281910,
        'action': 'click',
        'time': pd.Timestamp(year=2023, month=1, day=1, hour=12, minute=23, second=54),
    }
    
    custom_compare(parse_user_action_solution(example_1_line), example_1_res)
    
    example_2_line = "('id': 1, 'action': 'search', 'time': '1997-12-31 23:23:59')"
    example_2_res = {
        'id': 1,
        'action': 'search',
        'time': pd.Timestamp(year=1997, month=12, day=31, hour=23, minute=23, second=59),
    }
    
    custom_compare(parse_user_action_solution(example_2_line), example_2_res)
    
    print('Все тесты прошли успешно!')

In [66]:
parse_user_action_test()

Все тесты прошли успешно!


# Предобработка данных в pandas

In [76]:
import json

In [67]:
user_session = pd.read_csv('user_session_sample.csv')

In [68]:
user_session.head()

,id,user_id,ip,action_time,action
0,619,NaN,244.143.138.0,2021-01-20 18:36:32,"('id': 234676, 'action': 'search', 'time': '20..."
1,919428,30083.0,157.169.233.89,2023-12-10 04:13:50,"('id': 541654, 'action': 'search', 'time': '20..."
2,630539,NaN,160.111.52.224,2023-09-26 04:09:44,"('id': 230556, 'action': 'click', 'time': '202..."
3,963478,NaN,247.14.235.202,2023-12-19 21:02:47,"('id': 838067, 'action': 'click', 'time': '202..."
4,421566,NaN,255.2.67.154,2023-07-20 16:04:14,"('id': 924754, 'action': 'search', 'time': '20..."


In [70]:
user_session.isna().sum()

id                 0
user_id        66371
ip                 0
action_time    13922
action             0
dtype: int64

In [73]:
user_session.dtypes

id               int64
user_id        float64
ip              object
action_time     object
action          object
dtype: object

In [74]:
user_session['action_time'] = pd.to_datetime(user_session['action_time'])

In [86]:
parse_user_action_solution(user_session['action'][0])

{'id': 234676, 'action': 'search', 'time': Timestamp('2021-01-20 18:36:32')}

In [91]:
user_session['action'] = user_session['action'].apply(parse_user_action_solution)

In [100]:
user_session['action'][0]

{'id': 234676, 'action': 'search', 'time': Timestamp('2021-01-20 18:36:32')}

In [109]:
user_session['action_time'] = user_session['action'].apply(lambda x: x['time'])

In [110]:
user_session

,id,user_id,ip,action_time,action,time
0,619,NaN,244.143.138.0,2021-01-20 18:36:32,"{'id': 234676, 'action': 'search', 'time': 202...",2021-01-20 18:36:32
1,919428,30083.0,157.169.233.89,2023-12-10 04:13:50,"{'id': 541654, 'action': 'search', 'time': 202...",2023-12-10 04:13:50
2,630539,NaN,160.111.52.224,2023-09-26 04:09:44,"{'id': 230556, 'action': 'click', 'time': 2023...",2023-09-26 04:09:44
3,963478,NaN,247.14.235.202,2023-12-19 21:02:47,"{'id': 838067, 'action': 'click', 'time': 2023...",2023-12-19 21:02:47
4,421566,NaN,255.2.67.154,2023-07-20 16:04:14,"{'id': 924754, 'action': 'search', 'time': 202...",2023-07-20 16:04:14
...,...,...,...,...,...,...
100256,787682,NaN,235.213.10.189,2023-11-08 03:26:14,"{'id': 611537, 'action': 'click', 'time': 2023...",2023-11-08 03:26:14
100257,467742,95044.0,11.197.24.55,2023-08-07 18:09:05,"{'id': 105315, 'action': 'search', 'time': 202...",2023-08-07 18:09:05
100258,756486,NaN,25.201.68.226,2023-10-30 11:55:48,"{'id': 736403, 'action': 'search', 'time': 202...",2023-10-30 11:55:48
100259,585901,NaN,185.35.134.140,2023-09-14 19:06:45,"{'id': 614891, 'action': 'search', 'time': 202...",2023-09-14 19:06:45


In [189]:
((user_session['action_time'].dt.year == 2023) & (user_session['action_time'].dt.month == 8)).sum()

8395

## Задание 1

Подсчитайте, сколько пропущенных значений содержится в каждой из колонок таблицы.

## Задание 2

Известно, что значение в колонке «action_time» должно совпадать со значением `action_time`, которое хранится внутри значений характеристики «action».

Воспользуйтесь реализованной ранее функцией `parse_user_action_solution`, чтобы восстановить на основе значений из колонки «action» отсутствующую информацию о моментах времени, когда пользователи совершали соответствующие действия.

В качестве ответа укажите число пользовательских действий, которые были совершены в августе 2023-го года, рассчитанное по таблице с восстановленными данными.

# Исследование данных

## Задание 1

In [118]:
def round_to_2(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.
    
    Аргументы:
        x: Число.
        
    Возвращаемое значение:
        Результат округления числа до 2 знаков после запятой.
    """
    
    return round(x, 2)

In [119]:
def get_sessions(action_times, max_delta=40 * 60):
    """
    Разбивает список моментов времени, когда пользователь проявлял активность,
    на пользовательские сессии. 
    
    Аргументы:
        action_times: Список моментов времени, когда пользователь проявлял активность.
                      Гарантируется, что моменты времени упорядочены в порядке возрастания.
           max_delta: Максимальное значение (в секундах) промежутка между двумя активностями пользователя,
                      при котором они считаются относящимися к одной сессии.
                      
    Возвращаемое значение:
        Список пользовательских сессий. Каждая сессия представляется упорядоченным по возрастанию
        списком моментов времени для действий, которые были совершены в эту сессию.
    """
    
    sessions = []
    cur_session = []
    
    prev_time = None
    
    for time in action_times:
        if prev_time is None or (time - prev_time).total_seconds() > max_delta:
            if len(cur_session) > 0:
                sessions.append(cur_session)
                
            cur_session = [time]
        else:
            cur_session.append(time)
            
        prev_time = time
        
    sessions.append(cur_session)
    
    return sessions

In [120]:
from datetime import datetime
action_times = [
    datetime(2023, 10, 1, 12, 0, 0),
    datetime(2023, 10, 1, 12, 5, 0),
    datetime(2023, 10, 1, 12, 10, 0),
    datetime(2023, 10, 1, 12, 55, 0),  # Пробел больше max_delta
    datetime(2023, 10, 1, 13, 0, 0),
]

In [ ]:
sessions = get_sessions(action_times)
for num in range(len(sessions)):
    print(sorted(sessions[num]))
x = (sessions[1][-1]- sessions[1][0])


[datetime.datetime(2023, 10, 1, 12, 0), datetime.datetime(2023, 10, 1, 12, 5), datetime.datetime(2023, 10, 1, 12, 10)]
[datetime.datetime(2023, 10, 1, 12, 55), datetime.datetime(2023, 10, 1, 13, 0)]


datetime.timedelta(seconds=300)

In [216]:
sorted(action_times)

[datetime.datetime(2023, 10, 1, 12, 0),
 datetime.datetime(2023, 10, 1, 12, 5),
 datetime.datetime(2023, 10, 1, 12, 10),
 datetime.datetime(2023, 10, 1, 12, 55),
 datetime.datetime(2023, 10, 1, 13, 0)]

In [217]:
def get_avg_session_time(action_times):
    """
    По списку моментов времени, когда пользователь проявлял активность,
    вычисляет среднюю продолжительность его пользовательской сессии
    с точки зрения времени.
    
    Аргументы:
        action_times: Список моментов времени, когда пользователь проявлял активность.
                      Гарантируется, что моменты времени упорядочены в порядке возрастания.
                      
    Возвращаемое значение:
        Средняя продолжительность пользовательской сессии **в секундах**,
        округлённая до $2$-х знаков после запятой.
    """
    len_sessions = 0
    count_ses = 0
    sessions = get_sessions(sorted(action_times))

    for num in range(0, len(sessions)):
        if len(sessions[num]) >= 2:
            len_sessions+= (sessions[num][-1]- sessions[num][0]).total_seconds()
            #print(len_sessions)
            count_ses+=1
            #print(count_ses)
    if (count_ses == 0):
        return 0
    return round_to_2((len_sessions/count_ses)/60)
    

In [179]:
example_1_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=41, second=21),
        pd.Timestamp(year=2023, month=1, day=13, hour=15, minute=41, second=21),
    ]

In [174]:
ex = get_sessions(example_1_action_times)

In [177]:
ex[0][-1] - ex[0][0]

Timedelta('0 days 00:23:46')

In [220]:
get_avg_session_time(example_1_action_times)

23.77

In [219]:
def get_avg_session_time_test():
    example_1_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=41, second=21),
        pd.Timestamp(year=2023, month=1, day=13, hour=15, minute=41, second=21),
    ]
    
    example_1_res = 23.77
    
    custom_compare(get_avg_session_time(example_1_action_times), example_1_res)
    
    example_2_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=13, hour=15, minute=41, second=21),
    ]
    
    example_2_res = 0
    
    custom_compare(get_avg_session_time(example_2_action_times), example_2_res)
    
    example_3_action_times = [
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=17, second=35),
        pd.Timestamp(year=2023, month=1, day=12, hour=15, minute=18, second=24),
        pd.Timestamp(year=2023, month=2, day=21, hour=9, minute=42, second=31),
        pd.Timestamp(year=2023, month=2, day=21, hour=10, minute=21, second=9),
        pd.Timestamp(year=2023, month=2, day=21, hour=10, minute=37, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=17, minute=37, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=18, minute=12, second=46),
        pd.Timestamp(year=2023, month=2, day=27, hour=18, minute=44, second=53),
        pd.Timestamp(year=2023, month=2, day=27, hour=19, minute=2, second=11),
        pd.Timestamp(year=2023, month=8, day=21, hour=10, minute=37, second=46),
    ]
    
    example_3_res = 46.83
    
    custom_compare(get_avg_session_time(example_3_action_times), example_3_res)
    
    print('Все тесты прошли успешно!')

In [221]:
get_avg_session_time_test()

Все тесты прошли успешно!


In [190]:
user_session.head()

,id,user_id,ip,action_time,action,time
0,619,NaN,244.143.138.0,2021-01-20 18:36:32,"{'id': 234676, 'action': 'search', 'time': 202...",2021-01-20 18:36:32
1,919428,30083.0,157.169.233.89,2023-12-10 04:13:50,"{'id': 541654, 'action': 'search', 'time': 202...",2023-12-10 04:13:50
2,630539,NaN,160.111.52.224,2023-09-26 04:09:44,"{'id': 230556, 'action': 'click', 'time': 2023...",2023-09-26 04:09:44
3,963478,NaN,247.14.235.202,2023-12-19 21:02:47,"{'id': 838067, 'action': 'click', 'time': 2023...",2023-12-19 21:02:47
4,421566,NaN,255.2.67.154,2023-07-20 16:04:14,"{'id': 924754, 'action': 'search', 'time': 202...",2023-07-20 16:04:14


In [222]:
avg_time = user_session.groupby('ip', as_index=False).agg({'action_time': get_avg_session_time})

In [213]:
avg_time = user_session.groupby('ip')['action_time'].agg(lambda x: get_avg_session_time(x.tolist())).reset_index()

In [223]:
avg_time

,ip,action_time
0,0.106.25.239,0.00
1,0.11.175.78,0.00
2,0.113.1.142,13.90
3,0.123.24.62,0.00
4,0.130.111.71,0.67
...,...,...
11619,99.83.159.45,19.63
11620,99.86.191.3,0.85
11621,99.90.75.33,0.00
11622,99.96.162.47,0.00


In [227]:
4628/11624

0.3981417756366139

In [226]:
avg_time[avg_time['action_time'] == 0]

,ip,action_time
0,0.106.25.239,0.0
1,0.11.175.78,0.0
3,0.123.24.62,0.0
5,0.131.39.17,0.0
7,0.134.245.109,0.0
...,...,...
11611,99.49.61.11,0.0
11615,99.67.77.86,0.0
11618,99.82.142.180,0.0
11621,99.90.75.33,0.0


In [232]:
avg_time[avg_time['action_time'] != 0]['action_time'].mean()

5.619685534591195

## Задание 2

Используя функцию `get_avg_session_time`, вычислите среднюю продолжительность пользовательской сессии для пользователей из таблицы `user_session`, с которой вы работали в рамках предыдущих упражнений. Для этого усредните значения средней продолжительности пользовательских сессий по всем пользователям.

**Обратите внимание**, что среднее можно вычислять как по всем пользователям вообще, так и по тем, у кого среднее время пользовательской сессии больше $0$. В рамках задания вычислите значение по всему множеству пользователей.

Аналогично лекции нужно считать, что пользователя уникальным образом идентифицирует IP-адрес, с которого он совершал действия.

Ответ округлите до $2$-х знаков после запятой.

## Задание 3

### Вопрос 1

У какой доли пользователей среднее значение пользовательской сессии равно $0$?

Ответ округлите до $2$-х знаков после запятой.

### Вопрос 2

Рассчитайте средннюю продолжительность пользовательской сессии только по пользователям, у которых средняя продолжительность пользовательской сессии больше $0$.

Ответ округлите до $2$-х знаков после запятой.